In [12]:
# Import libraries
from PIL import Image, UnidentifiedImageError
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing import image

In [13]:
# Define paths - Update these paths to your local dataset paths
isl_path = r"ISL_Dataset"
mnist_train_path = r"sign_mnist_train.csv"
mnist_test_path = r"sign_mnist_test.csv"
asl_path = r"asl_alphabet_train"

In [14]:
# Load ISL dataset function
def load_isl_dataset(isl_path):
    images_data = []
    labels = []
    for label in os.listdir(isl_path):
        label_folder = os.path.join(isl_path, label)
        if os.path.isdir(label_folder):
            for img_file in os.listdir(label_folder):
                img_path = os.path.join(label_folder, img_file)
                if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    with Image.open(img_path) as img:
                        img = img.convert('L')  # Convert to grayscale
                        img = img.resize((28, 28))
                        img_array = np.array(img).flatten()
                        images_data.append(img_array)
                        labels.append(label)
    return np.array(images_data), np.array(labels)

# Load ISL dataset
isl_images, isl_labels = load_isl_dataset(isl_path)

In [15]:
# Load and preprocess MNIST dataset
mnist_train = pd.read_csv(mnist_train_path)
mnist_test = pd.read_csv(mnist_test_path)
mnist_labels = mnist_train['label'].values
mnist_train.drop('label', axis=1, inplace=True)
mnist_images = np.array([np.reshape(i, (28, 28)) for i in mnist_train.values])
mnist_images = mnist_images.reshape(mnist_images.shape[0], 28, 28, 1)

# Reshape ISL images to match MNIST dimensions
isl_images = isl_images.reshape(isl_images.shape[0], 28, 28, 1)

# Combine MNIST and ISL datasets
combined_images = np.concatenate((mnist_images, isl_images), axis=0)
combined_labels = np.concatenate((mnist_labels, isl_labels), axis=0)

In [16]:
# Load ASL dataset function
def load_asl_dataset(asl_path):
    asl_images = []
    labels = []
    
    # Traverse each label folder inside the main ASL dataset directory
    for label in os.listdir(asl_path):
        folder_path = os.path.join(asl_path, label)
        
        # Check if the current item is a directory
        if os.path.isdir(folder_path):
            for img_file in os.listdir(folder_path):
                img_path = os.path.join(folder_path, img_file)
                
                # Load the image and handle exceptions
                try:
                    img = image.load_img(img_path, target_size=(28, 28), color_mode="grayscale")
                    img_array = image.img_to_array(img) / 255.0  # Normalize pixel values
                    asl_images.append(img_array)
                    labels.append(label)
                except UnidentifiedImageError:
                    print(f"Could not process file: {img_file}")
                except PermissionError:
                    print(f"Permission denied for file: {img_path}")
    
    return np.array(asl_images), np.array(labels)

# Load ASL dataset
asl_images, asl_labels = load_asl_dataset(asl_path)
asl_images = asl_images.reshape(asl_images.shape[0], 28, 28, 1)  # Reshape for CNN input

print("Dataset loaded successfully!")

Permission denied for file: asl_alphabet_train\asl_alphabet_train\A
Permission denied for file: asl_alphabet_train\asl_alphabet_train\B
Permission denied for file: asl_alphabet_train\asl_alphabet_train\C
Permission denied for file: asl_alphabet_train\asl_alphabet_train\D
Permission denied for file: asl_alphabet_train\asl_alphabet_train\E
Permission denied for file: asl_alphabet_train\asl_alphabet_train\F
Permission denied for file: asl_alphabet_train\asl_alphabet_train\G
Permission denied for file: asl_alphabet_train\asl_alphabet_train\H
Permission denied for file: asl_alphabet_train\asl_alphabet_train\I
Permission denied for file: asl_alphabet_train\asl_alphabet_train\J
Permission denied for file: asl_alphabet_train\asl_alphabet_train\K
Permission denied for file: asl_alphabet_train\asl_alphabet_train\L
Permission denied for file: asl_alphabet_train\asl_alphabet_train\M
Permission denied for file: asl_alphabet_train\asl_alphabet_train\N
Permission denied for file: asl_alphabet_train\a

In [17]:
# Combine ASL, ISL, and MNIST datasets
combined_images = np.concatenate((combined_images, asl_images), axis=0)
combined_labels = np.concatenate((combined_labels, asl_labels), axis=0)

# Transform labels to binary representation
label_binarizer = LabelBinarizer()
combined_labels = label_binarizer.fit_transform(combined_labels)

# Split data
x_train, x_test, y_train, y_test = train_test_split(combined_images, combined_labels, test_size=0.3, random_state=101)

In [18]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(x_train)

In [19]:
# Load MobileNetV2 model
mobilenet = MobileNetV2(input_shape=(96, 96, 3), include_top=False, weights='imagenet')
for layer in mobilenet.layers:
    layer.trainable = False

# Preprocess input data to match MobileNetV2 dimensions
x_train_resized = np.array([cv2.resize(cv2.merge([img] * 3), (96, 96)) for img in x_train])
x_test_resized = np.array([cv2.resize(cv2.merge([img] * 3), (96, 96)) for img in x_test])

# Add custom layers on top of MobileNetV2
model = Sequential([
    mobilenet,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_binarizer.classes_), activation='softmax')
])

# Compile model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train model
history = model.fit(datagen.flow(x_train_resized, y_train, batch_size=64), validation_data=(x_test_resized, y_test),
                    epochs=10, callbacks=[reduce_lr, early_stopping])

# Save model
model.save("combined_sign_language_mobilenet.h5")
print("Model Saved")

MemoryError: Unable to allocate 4.06 GiB for an array with shape (19709, 96, 96, 3) and data type float64

In [ ]:
# Plot accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Accuracy")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'])
plt.show()

NameError: name 'history' is not defined

In [ ]:
# Evaluation
y_pred = model.predict(x_test_resized)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print("Accuracy:", accuracy_score(y_true, y_pred_classes))
print(confusion_matrix(y_true, y_pred_classes))
print(classification_report(y_true, y_pred_classes))